<a href="https://colab.research.google.com/github/NicCusinato/Webscraper/blob/main/Webscraper_3_0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Webscraper 3.0 is used to clean up the output and give you one file showing the missing dropdowns on the main page compared to the others**

# Setting up Selenium Enviornment

In [1]:
import os
import time
!apt-get update
!apt install chromium-browser
!pip install google-colab-selenium
!pip install chromedriver-autoinstaller
import chromedriver_autoinstaller
import google_colab_selenium as gs

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Get:3 https://dl.google.com/linux/chrome/deb stable InRelease [1,825 B]
Hit:4 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:5 http://archive.ubuntu.com/ubuntu jammy InRelease
Ign:6 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:7 https://r2u.stat.illinois.edu/ubuntu jammy Release
Hit:8 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:9 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Get:10 https://dl.google.com/linux/chrome/deb stable/main amd64 Packages [1,086 B]
Hit:11 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:12 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:13 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Fetched 2,911 B in 4s (653 B/s)
Reading package lists... Done
W: Skipping a

#Run Enviornment

In [6]:
# Set up your WebDriver (make sure you have the correct path to your chromedriver)
driver = gs.Chrome()

<IPython.core.display.Javascript object>

#Scrape Pages

In [31]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import ElementNotInteractableException, TimeoutException, NoSuchElementException
import time


# Function to scrape dropdown (listbox) options from a page
def get_custom_dropdown_options(url):
    driver.get(url)
    time.sleep(2)  # Give the page some time to load fully

    # Scroll the entire page to ensure all lazy-loaded content is rendered
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(2)  # Wait for page scroll and any lazy-loaded content to render
    driver.execute_script("window.scrollTo(0, 0);")

    # Find all button elements that act as listboxes
    dropdown_buttons = driver.find_elements(By.CSS_SELECTOR, 'button[aria-haspopup="listbox"]')
    options_dict = {}
    missed_dropdowns = []

    # Loop through each dropdown button and extract options
    for index, button in enumerate(dropdown_buttons):
        try:
            # Scroll into view if necessary
            driver.execute_script("arguments[0].scrollIntoView(true);", button)
            time.sleep(0.5)  # Small delay to ensure smooth scrolling

            # Get the label of the dropdown, if available
            label_element = button.find_element(By.CLASS_NAME, 'swan-listbox-button-label')
            dropdown_label = label_element.text if label_element else f"Unnamed dropdown {index + 1}"

            # Click the button to expand the dropdown (listbox)
            WebDriverWait(driver, 10).until(EC.element_to_be_clickable(button)).click()
            time.sleep(1)  # Wait for the dropdown to expand

            # Find all the options within the listbox
            option_elements = driver.find_elements(By.CSS_SELECTOR, 'li[role="option"]')

            option_values = []
            for option in option_elements:
                data_key = option.get_attribute("data-key")  # Get the data-key attribute
                visible_text = option.find_element(By.CLASS_NAME, 'swan-listbox-option-text-container').text  # Get the visible text
                option_values.append({
                    "data_key": data_key,
                    "visible_text": visible_text,
                    "source_url": url  # Track the source URL for each option
                })

            # Store the dropdown label and its associated options
            options_dict[dropdown_label] = option_values

            # Click somewhere outside the dropdown to close it (e.g., the body of the page)
            driver.find_element(By.TAG_NAME, 'body').click()
            time.sleep(0.5)  # Ensure the dropdown is fully closed before moving on
        except (ElementNotInteractableException, TimeoutException, NoSuchElementException) as e:
            print(f"Could not interact with dropdown {index + 1}: {e}")
            missed_dropdowns.append(f"Dropdown {index + 1}: {e}")
        except Exception as e:
            print(f"An unexpected error occurred: {e}")
            missed_dropdowns.append(f"Dropdown {index + 1}: {e}")

    return options_dict, missed_dropdowns

# Save dropdown options to a file
def save_to_file(filename, options_dict):
    with open(filename, 'w') as f:
        for dropdown_label, options in options_dict.items():
            f.write(f"Dropdown: {dropdown_label}\n")
            for option in options:
                f.write(f"Option: {option['visible_text']} (data-key: {option['data_key']})\n")
            f.write("\n")

# Save missed dropdowns to a file for troubleshooting
def save_missed_to_file(filename, missed_dropdowns):
    with open(filename, 'w') as f:
        for error_message in missed_dropdowns:
            f.write(f"{error_message}\n")

# Modify the compare_options function to include all sources for missing options
def compare_options(base_options, other_options):
    missing_in_base = {}
    missing_in_other = {}
    option_sources = {}

    # Collect sources for options in other pages
    for dropdown_label, options in other_options.items():
        for option in options:
            data_key = option['data_key']
            if dropdown_label not in option_sources:
                option_sources[dropdown_label] = {}
            if data_key not in option_sources[dropdown_label]:
                option_sources[dropdown_label][data_key] = []
            option_sources[dropdown_label][data_key].append(option['source_url'])

    # Add sources for base options to option_sources
    for dropdown_label, options in base_options.items():
        for option in options:
            data_key = option['data_key']
            if dropdown_label not in option_sources:
                option_sources[dropdown_label] = {}
            if data_key not in option_sources[dropdown_label]:
                option_sources[dropdown_label][data_key] = []
            option_sources[dropdown_label][data_key].append(option['source_url'])

    # Compare options and find missing ones
    for dropdown_label, options in other_options.items():
        base_option_keys = {opt['data_key'] for opt in base_options.get(dropdown_label, [])}
        other_option_keys = {opt['data_key'] for opt in options}

        # Find options that exist in other but not in base
        missing_in_base[dropdown_label] = {key: option_sources[dropdown_label][key]
                                            for key in other_option_keys - base_option_keys}

        # Find options that exist in base but not in other
        missing_in_other[dropdown_label] = {key: option_sources[dropdown_label][key]
                                             for key in base_option_keys - other_option_keys}

    return missing_in_base, missing_in_other

# Save missing options to a file with all sources
with open('missing_options.txt', 'w') as f:
    f.write("Options in other pages but not in the base page:\n\n")
    for dropdown_label, missing_options in missing_in_base.items():
        if missing_options:
            f.write(f"Dropdown: {dropdown_label}\n")
            for option_key, sources in missing_options.items():
                formatted_option_key = option_key.replace("&quote;", '"')
                f.write(f"Missing Option (data-key): {formatted_option_key} found in pages: {', '.join(sources)}\n")
            f.write("\n")

    f.write("Options in the base page but not in other pages:\n\n")
    for dropdown_label, missing_options in missing_in_other.items():
        if missing_options:
            f.write(f"Dropdown: {dropdown_label}\n")
            for option_key, sources in missing_options.items():
                formatted_option_key = option_key.replace("&quote;", '"')
                f.write(f"Missing Option (data-key): {formatted_option_key} found in base page\n")
            f.write("\n")
# Example usage
base_url = "https://www.vistaprint.com/labels-stickers/custom-stickers/single-stickers"
base_dropdown_options, missed_dropdowns = get_custom_dropdown_options(base_url)

# Save base page options to a file
save_to_file('base_page_options.txt', base_dropdown_options)

# Save missed dropdowns to a separate file
if missed_dropdowns:
    save_missed_to_file('missed_dropdowns.txt', missed_dropdowns)

# Queue of other URLs to compare against the base URL
other_urls = [
    "https://www.vistaprint.com/labels-stickers/custom-stickers/die-cut-sticker-singles",
    "https://www.vistaprint.com/labels-stickers/custom-stickers/holographic-stickers",
    "https://www.vistaprint.com/labels-stickers/custom-stickers/face-stickers",
    "https://www.vistaprint.com/labels-stickers/custom-stickers/party-stickers",
    "https://www.vistaprint.com/labels-stickers/custom-stickers/reusable-stickers",
    "https://www.vistaprint.com/labels-stickers/custom-stickers/campaign-stickers",
    "https://www.vistaprint.com/labels-stickers/qr-code-stickers",
    "https://www.vistaprint.com/labels-stickers/custom-stickers/kiss-cut-stickers",
    "https://www.vistaprint.com/labels-stickers/custom-stickers/front-adhesive-stickers",
    "https://www.vistaprint.com/labels-stickers/custom-stickers/soft-touch-stickers"
]

# Scrape options from each URL in the queue
all_other_options = {}
for other_url in other_urls:
    options, missed = get_custom_dropdown_options(other_url)
    all_other_options.update(options)

# Compare options
missing_in_base, missing_in_other = compare_options(base_dropdown_options, all_other_options)

# Save missing options to a file
with open('missing_options.txt', 'w') as f:
    f.write("Options in other pages but not in the base page:\n\n")
    for dropdown_label, missing_options in missing_in_base.items():
        if missing_options:
            f.write(f"Dropdown: {dropdown_label}\n")
            for option_key, sources in missing_options.items():
                f.write(f"Missing Option (data-key): {option_key} found in pages: {', '.join(sources)}\n")
            f.write("\n")

    f.write("Options in the base page but not in other pages:\n\n")
    for dropdown_label, missing_options in missing_in_other.items():
        if missing_options:
            f.write(f"Dropdown: {dropdown_label}\n")
            for option_key, sources in missing_options.items():
                f.write(f"Missing Option (data-key): {option_key} found in base page\n")
            f.write("\n")




Could not interact with dropdown 4: Message: no such element: Unable to locate element: {"method":"css selector","selector":".swan-listbox-option-text-container"}
  (Session info: chrome=129.0.6668.89); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
#0 0x58eaaaa8102a <unknown>
#1 0x58eaaa7675e0 <unknown>
#2 0x58eaaa7b6be8 <unknown>
#3 0x58eaaa7b6e81 <unknown>
#4 0x58eaaa7ab586 <unknown>
#5 0x58eaaa7dbb4d <unknown>
#6 0x58eaaa7ab478 <unknown>
#7 0x58eaaa7dbcee <unknown>
#8 0x58eaaa7fad7d <unknown>
#9 0x58eaaa7db8c3 <unknown>
#10 0x58eaaa7a96b3 <unknown>
#11 0x58eaaa7aa68e <unknown>
#12 0x58eaaaa4ba2b <unknown>
#13 0x58eaaaa4f9b1 <unknown>
#14 0x58eaaaa38225 <unknown>
#15 0x58eaaaa50532 <unknown>
#16 0x58eaaaa1d38f <unknown>
#17 0x58eaaaa6ff28 <unknown>
#18 0x58eaaaa700f3 <unknown>
#19 0x58eaaaa7fe7c <unknown>
#20 0x7c7debf65ac3 <unknown>

Could not interact with dropdown 2: Mess

# Close Enviornment


In [ ]:
# Close the WebDriver
driver.quit()